# Random Forest
No model training here.
Just plot the numbers from RF_107.

In [1]:
from datetime import datetime
print(datetime.now())

2023-06-05 17:25:02.955336


In [2]:
import tensorflow as tf
dt='float32'
tf.keras.backend.set_floatx('float32')
tf.random.set_seed(42) # supposedly leads to reproducible results

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found')
else:
    print('Found GPU at: {}'.format(device_name))

try:
    from google.colab import drive
    IN_COLAB = True
    print('Running on CoLab')
    PATH='/content/drive/'
    drive.mount(PATH)
    DATA_DIR=PATH+'My Drive/data/IRP2/'  # must end in "/"
    MODEL_DIR=PATH+'My Drive/data/IRP2/Models/'  # must end in "/"
except:
    IN_COLAB = False
    print('Running on Mac')
    DATA_DIR="/Users/jasonmiller/WVU/BAM_ML/"
    MODEL_DIR="/Users/jasonmiller/WVU/BAM_ML/Models/"
print(DATA_DIR)
SAVE_MODEL_FILENAME = None 

2023-06-05 17:25:03.009944: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


GPU device not found
Running on Mac
/Users/jasonmiller/WVU/BAM_ML/


2023-06-05 17:25:12.407921: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from platform import python_version
print('Python',python_version())
import numpy as np
np.random.seed(42) # supposedly sets scikit-learn
import pandas as pd  # for plotting
import time # sleep function
from os.path import isfile
from matplotlib import pyplot as plt 
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)

Python 3.10.0
sklearn 1.1.2


## Data Load

In [4]:
print(datetime.now())
filename = DATA_DIR+'RF_107.csv'
data = []
with open (filename,'r') as fin:
    for line in fin:
        line=line.strip()
        fields=line.split(',')
        true_label = int(fields[0])
        pred_label = int(fields[1])
        rfc_prob   = float(fields[2])
        data.append( (true_label,pred_label,rfc_prob) )
print('Values loaded: ', len(data))
print(datetime.now())

2023-06-05 17:25:14.541295
Values loaded:  400000
2023-06-05 17:25:15.123702


In [5]:
sdata = sorted(data, key=lambda x: x[2])
print('Values sorted: ', len(sdata))
print(datetime.now())
data = None

Values sorted:  400000
2023-06-05 17:25:15.276273


In [6]:
def find_threshold(sdata,left_to_right=True,FDR=0.05):
    '''sdata: list of tuples of (true, predicted, probability)
               sorted by last field increasing
    '''
    length = len(sdata)
    if left_to_right:
        start_pos = 0
        stop_pos = length
        step_size = 1
    else:
        start_pos = length-1
        stop_pos = -1
        step_size = -1
    total_seen = 0
    total_incorrect = 0
    portion_incorrect = 0.0
    last_pos_under_fdr = 0
    last_prob_under_fdr = 0
    for pos in range(start_pos,stop_pos,step_size):
        true_label,pred_label,pred_prob = sdata[pos]
        total_seen += 1
        if true_label != pred_label: 
            total_incorrect += 1
        portion_incorrect = 1.0*total_incorrect/total_seen
        if portion_incorrect <= FDR:
            last_pos_under_fdr = pos
            last_prob_under_fdr = pred_prob
    return last_pos_under_fdr, 100.0*last_prob_under_fdr

In [9]:
length = len(sdata)
FDR=0.05

last_pos,last_prob = find_threshold(sdata,True,FDR)
portion = 100.0*last_pos/length
print('To choose true M reads with %4.2f FDR,'%FDR)
print('choose reads with RF probability score %4.2f%% or less.'%last_prob)
print('That will retain %4.2f%% of reads.'%portion)

print()

last_pos, last_prob = find_threshold(sdata,False,FDR)
portion = 100.0*(length-last_pos)/length
print('To choose true S reads with %4.2f FDR,'%FDR)
print('choose reads with RF probability score %4.2f%% or more.'%last_prob)
print('That will retain %4.2f%% of reads.'%portion)


To choose true M reads with 0.05 FDR,
choose reads with RF probability score 22.00% or less.
That will retain 35.28% of reads.

To choose true S reads with 0.05 FDR,
choose reads with RF probability score 87.00% or more.
That will retain 31.23% of reads.
